## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1742191303046, experiment_id='0', last_update_time=1742191303046, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/03/17 11:02:27 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/03/17 11:02:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/a3afe770ff714fce8d5e65073529ec45/artifacts'
🏃 View run sneaky-fowl-960 at: http://127.0.0.1:5000/#/experiments/1/runs/a3afe770ff714fce8d5e65073529ec45
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1742191347510, experiment_id='1', last_update_time=1742191347510, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1742191303046, experiment_id='0', last_update_time=1742191303046, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_registered_models()

[]

In [9]:
client = MlflowClient()
experiment_id = "1"  # Make sure this is the correct experiment ID

In [10]:
# ✅ Get the latest run_id
latest_run = client.search_runs(
    experiment_ids=[experiment_id], order_by=["start_time desc"], max_results=1
)

if latest_run and len(latest_run) > 0:
    run_id = latest_run[0].info.run_id
    print(f"Latest run_id: {run_id}")
else:
    print("No runs found for the given experiment.")

Latest run_id: a3afe770ff714fce8d5e65073529ec45
